AI Support Chatbot for Plant Disease Diagnosis
● Objective:
Create an AI-powered chatbot to automate responses to plant disease-related queries.

● Approach:
Used NLP techniques (TF-IDF + Logistic Regression) for intent classification and integrated Google Gemini for fallback responses. Implemented the chatbot in Python and adapted it for Jupyter/Colab for easy testing.

● Skills Learned:
○ NLP for conversational agents
○ Intent classification using machine learning
○ Integration of generative AI (Gemini)
○ Chatbot interaction handling in Python environments

In [3]:
import re
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from joblib import dump, load
import google.generativeai as genai

# Download tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')

# Gemini API Configuration
genai.configure(api_key="your_actual_key")
model = genai.GenerativeModel("gemini-1.5-flash")

intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
        "responses": ["Hello! How can I assist you today?", "Hi there!", "Hey! What's on your mind?"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "Goodbye", "See you later", "Take care"],
        "responses": ["Goodbye! Have a great day!", "See you later!", "Take care and stay safe!"]
    },
    {
        "tag": "powdery_mildew",
        "patterns": [
            "What is powdery mildew?", "How do I treat powdery mildew?", "What causes powdery mildew?",
            "How to identify powdery mildew on leaves", "Is powdery mildew harmful?", "What plants are affected by powdery mildew?",
            "What causes powdery mildew disease?", "Symptoms of powdery mildew",
            "What is powdermildew?", "How do I treat powdermildew?", "What causes powdermildew?",
            "How to identify powdermildew", "Is powdermildew harmful?", "Symptoms of powdermildew"
        ],
        "responses": [
            "Powdery mildew is a fungal disease that causes white, powdery spots on leaves. It thrives in humid conditions. To treat it, use sulfur-based fungicides and improve airflow around your plants."
        ]
    },
    {
        "tag": "root_rot",
        "patterns": [
            "What is root rot?", "How do I treat root rot?", "What causes root rot?",
            "How do I fix root rot in plants", "Can root rot be reversed?", "Signs of root rot in plants",
            "My plant's roots are black and mushy, is it root rot?", "How do I know if my plant has root rot?",
            "What is rootrot?", "How do I treat rootrot?", "What causes rootrot?",
            "How to fix rootrot", "Can rootrot be reversed?", "Signs of rootrot"
        ],
        "responses": [
            "Root rot is caused by overwatering and poor drainage, leading to black, mushy roots. To fix root rot, remove the affected roots and repot the plant in well-draining soil. Avoid overwatering."
        ]
    },
    {
        "tag": "leaf_spot",
        "patterns": [
            "What is leaf spot?", "How do I treat leaf spot?", "What causes leaf spot?",
            "Is leaf spot harmful?", "How do I prevent leaf spot?", "Why are my plant leaves spotted?",
            "My plant has black spots on its leaves, what is it?", "What is causing dark spots on my plant leaves?",
            "What is leafspot?", "How do I treat leafspot?", "What causes leafspot?",
            "How to prevent leafspot", "Why are my plant leaves spotted?", "How to treat leafspot"
        ],
        "responses": [
            "Leaf spot is a fungal or bacterial disease that causes dark spots or lesions on leaves. To treat it, remove affected leaves and use fungicides. Ensure good air circulation around the plant."
        ]
    },
    {
        "tag": "blight",
        "patterns": [
            "What is blight?", "How do I treat blight?", "What causes blight?",
            "Can blight spread?", "How to prevent blight in plants",
            "My plant is wilting rapidly, could it be blight?", "How to tell if my plant has blight?",
            "What is blight disease?", "How do I treat blight disease?", "What causes blight disease?",
            "How to prevent blight disease?"
        ],
        "responses": [
            "Blight is a plant disease that causes rapid wilting and browning of plant tissues. It can be caused by bacteria, fungi, or environmental stress. Prune affected areas and apply fungicides."
        ]
    },
    {
        "tag": "aphids",
        "patterns": [
            "What are aphids?", "How do I treat aphids?", "What damage do aphids cause?",
            "How to control aphids on plants", "Are aphids harmful to my plants?",
            "My plant's leaves are curling and yellowing, could it be aphids?", "How do I know if I have aphids on my plants?",
            "What are aphid pests?", "How do I treat aphid pests?", "What damage do aphid pests cause?",
            "How to control aphid pests on plants"
        ],
        "responses": [
            "Aphids are small, green insects that suck sap from plants, causing stunted growth and yellowing leaves. Treat aphid infestations with insecticidal soap or neem oil."
        ]
    },
    {
        "tag": "whiteflies",
        "patterns": [
            "What are whiteflies?", "How to get rid of whiteflies?", "What damage do whiteflies cause?",
            "How to treat whiteflies in plants", "Why are my plants infested with whiteflies?",
            "My plant's leaves are turning yellow, could whiteflies be the cause?", "How do I control whiteflies on my plants?",
            "What are white fly pests?", "How to get rid of white fly pests?", "What damage do white fly pests cause?",
            "How to control white fly pests"
        ],
        "responses": [
            "Whiteflies are tiny insects that suck sap from plants, leading to yellowing leaves. Control whiteflies using insecticidal soap or by introducing natural predators like ladybugs."
        ]
    },
    {
        "tag": "fungal_infections",
        "patterns": [
            "What is a fungal infection?", "How do I treat fungal infections on plants?",
            "What causes fungal infections in plants?", "Can fungal infections be prevented?",
            "How to control fungal infections in plants?",
            "My plant has moldy growth and wilting leaves, could it be a fungal infection?", "How do I treat fungal growth on my plants?",
            "What is fungalinfection?", "How do I treat fungalinfection?", "What causes fungalinfection?",
            "How to control fungalinfection in plants"
        ],
        "responses": [
            "Fungal infections on plants can cause a range of symptoms like moldy growth, spots, and wilting. Treat fungal infections with fungicides and remove affected parts of the plant."
        ]
    },
    {
        "tag": "mosaic_virus",
        "patterns": [
            "What is mosaic virus?", "How do I treat mosaic virus?", "What causes mosaic virus?",
            "Can mosaic virus spread?", "How to identify mosaic virus on plants",
            "My plant leaves have yellow streaks, could it be mosaic virus?", "How to spot mosaic virus on leaves?",
            "What is mosaicvirus?", "How do I treat mosaicvirus?", "What causes mosaicvirus?",
            "How to identify mosaicvirus"
        ],
        "responses": [
            "Mosaic virus causes yellow or light green streaks on plant leaves, often in a mosaic pattern. The virus is typically spread by insects like aphids. Remove infected plants and control insect pests to prevent its spread."
        ]
    },
    {
        "tag": "rust_fungus",
        "patterns": [
            "What is rust fungus?", "How do I treat rust fungus?", "What causes rust fungus?",
            "How to prevent rust fungus on plants", "Is rust fungus harmful?",
            "How do I identify rust fungus?", "My plant has orange or yellow spots on the leaves, could it be rust fungus?",
            "How to deal with rust fungus?",
            "What is rustfungus?", "How do I treat rustfungus?", "What causes rustfungus?",
            "How to prevent rustfungus", "How do I identify rustfungus?"
        ],
        "responses": [
            "Rust fungus appears as orange, yellow, or red spots on the leaves of plants. It can cause stunted growth and yellowing. To treat it, remove infected leaves and apply fungicides."
        ]
    },
    {
        "tag": "fusarium_wilt",
        "patterns": [
            "What is fusarium wilt?", "How do I treat fusarium wilt?", "What causes fusarium wilt?",
            "How to prevent fusarium wilt", "Is fusarium wilt a fungus?", "Symptoms of fusarium wilt",
            "My plant is wilting and the leaves are yellowing, could it be fusarium wilt?", "How to stop fusarium wilt from spreading?",
            "What is fusariumwilt?", "How do I treat fusariumwilt?", "What causes fusariumwilt?",
            "How to prevent fusariumwilt", "My plant is wilting and the leaves are yellowing, could it be fusariumwilt?"
        ],
        "responses": [
            "Fusarium wilt is caused by a soil-borne fungus that blocks the plant's ability to absorb water, leading to wilting and yellowing leaves. There is no cure, so it's best to remove infected plants and avoid replanting in the same soil."
        ]
    },
    {
        "tag": "bacterial_wilt",
        "patterns": [
            "What is bacterial wilt?", "How do I treat bacterial wilt?", "What causes bacterial wilt?",
            "Is bacterial wilt harmful?", "How to prevent bacterial wilt", "How to identify bacterial wilt?",
            "My plant suddenly wilted and died, could it be bacterial wilt?", "What are the signs of bacterial wilt?",
            "What is bacterialwilt?", "How do I treat bacterialwilt?", "What causes bacterialwilt?",
            "How to prevent bacterialwilt", "My plant suddenly wilted and died, could it be bacterialwilt?"
        ],
        "responses": [
            "Bacterial wilt is caused by bacteria that infect the vascular system of plants, causing wilting and sudden death. There is no cure, but you can prevent its spread by removing infected plants and disinfecting tools."
        ]
    },
    {
        "tag": "anthracnose",
        "patterns": [
            "What is anthracnose?", "How do I treat anthracnose?", "What causes anthracnose?",
            "How to prevent anthracnose in plants", "Symptoms of anthracnose on plants",
            "My plant has sunken black lesions, is it anthracnose?", "How do I control anthracnose?",
            "What is anthracnose disease?", "How do I treat anthracnose disease?", "What causes anthracnose disease?"
        ],
        "responses": [
            "Anthracnose is a fungal disease that causes dark, sunken lesions on leaves, stems, and fruits. To treat it, prune infected parts and apply fungicides. Good air circulation can help prevent it."
        ]
    },
    {
        "tag": "verticillium_wilt",
        "patterns": [
            "What is verticillium wilt?", "How do I treat verticillium wilt?", "What causes verticillium wilt?",
            "Is verticillium wilt fatal?", "How to prevent verticillium wilt in plants",
            "My plant is wilting and yellowing, could it be verticillium wilt?", "How to manage verticillium wilt?",
            "What is verticilliumwilt?", "How do I treat verticilliumwilt?", "What causes verticilliumwilt?",
            "How to manage verticilliumwilt"
        ],
        "responses": [
            "Verticillium wilt is a soil-borne fungal disease that disrupts water uptake, causing wilting and yellowing. There's no cure, but removing infected plants and rotating crops can help manage it."
        ]
    },
    {
        "tag": "gray_mold",
        "patterns": [
            "What is gray mold?", "How do I treat gray mold?", "What causes gray mold?",
            "How to prevent gray mold in plants", "Is gray mold harmful to plants?",
            "My plant has fuzzy gray spots on leaves, could it be gray mold?", "How to treat gray mold on plants?",
            "What is graymold?", "How do I treat graymold?", "What causes graymold?",
            "How to prevent graymold in plants", "Is graymold harmful to plants?"
        ],
        "responses": [
            "Gray mold appears as fuzzy gray growth on leaves and flowers, especially in humid conditions. Treat it by pruning affected areas and using fungicides. Improve ventilation to reduce humidity."
        ]
    },
    {
        "tag": "nematode_wilt",
        "patterns": [
            "What is nematode wilt?", "How do I treat nematode wilt?", "What causes nematode wilt?",
            "How to prevent nematode wilt", "Is nematode wilt a fungal disease?",
            "My plant roots are damaged and it’s wilting, could it be nematode wilt?", "How to control nematode wilt?",
            "What is nematodewilt?", "How do I treat nematodewilt?", "What causes nematodewilt?",
            "How to control nematodewilt", "My plant roots are damaged and it’s wilting, could it be nematodewilt?"
        ],
        "responses": [
            "Nematode wilt is caused by microscopic worms called nematodes that attack plant roots, preventing them from absorbing nutrients. To manage it, use nematode-resistant varieties and practice crop rotation."
        ]
    }
]

# Prepare training data
training_sentences = []
training_labels = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Vectorize and train
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
X_train = vectorizer.fit_transform(training_sentences)
classifier = LogisticRegression()
classifier.fit(X_train, training_labels)

# Save and reload (optional, helpful in reuse)
dump({"vectorizer": vectorizer, "classifier": classifier}, "intent_model.joblib")
data = load("intent_model.joblib")
vectorizer = data["vectorizer"]
classifier = data["classifier"]

# Helper functions
def predict_intent(sentence):
    sentence_vectorized = vectorizer.transform([sentence])
    predicted_label = classifier.predict(sentence_vectorized)
    return predicted_label[0]

def get_response(user_input):
    predicted_tag = predict_intent(user_input)
    return random.choice(responses[predicted_tag])

def clean_gemini_output(text):
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    return re.sub(r'\*(.*?)\*', r'\1', text)

def fallback_to_gemini(query):
    response = model.generate_content(query)
    return clean_gemini_output(response.text)

# Main interaction loop
last_question = None

print("🌿 Plant Disease Chatbot (type 'exit' to quit, or 'use gemini' for fallback)\n")
while True:
    user_input = input("You: ").strip()

    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye! 🌱")
        break
    elif user_input.lower() == 'use gemini':
        if last_question:
            gemini_response = fallback_to_gemini(last_question)
            print(f"Gemini: {gemini_response}")
        else:
            print("Chatbot: No previous question to send to Gemini.")
    else:
        try:
            response = get_response(user_input)
            print(f"Chatbot: {response}")
        except:
            print("Chatbot: I'm not sure. You can type 'use gemini' for more info.")
        last_question = user_input


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


🌿 Plant Disease Chatbot (type 'exit' to quit, or 'use gemini' for fallback)

You: hello
Chatbot: Hey! What's on your mind?
You: what is nematode wilt?
Chatbot: Nematode wilt is caused by microscopic worms called nematodes that attack plant roots, preventing them from absorbing nutrients. To manage it, use nematode-resistant varieties and practice crop rotation.
You: use gemini
Gemini: Nematode wilt isn't a single disease with one cause.  Instead, it refers to wilting in plants caused by root-knot nematodes (genus Meloidogyne) or other nematodes that damage the plant's root system. These nematodes aren't directly causing the wilting in the same way a fungus or bacteria would through a specific toxin.  Instead, their feeding activity on the roots causes:

* Root damage: Nematodes puncture the roots, creating feeding sites called galls or knots. This disrupts the flow of water and nutrients to the plant.
* Reduced root function: The damaged roots are less able to absorb water and nutrient